In [4]:
!pip install pycountry
!pip install catboost

In [5]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry as pc

pd.options.display.max_columns = None

In [6]:
## Importing Data
data = pd.read_csv('./hotel_bookings.csv')

In [7]:
## Show the first 5 rows of Data
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [8]:
data.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [9]:
## Copy the dataset
df = data.copy()

In [10]:
## Find the missing value, show the total null values for each column and sort it in descending order
df.isnull().sum().sort_values(ascending=False)[:10]

company                     112593
agent                        16340
country                        488
children                         4
lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
is_canceled                      0
market_segment                   0
dtype: int64

### agent,company,country,children

In [11]:
## Drop Rows where there is no adult, baby and child
df = df.drop(df[(df.adults+df.babies+df.children)==0].index)


## If no id of agent or company is null, just replace it with 0
df[['agent','company']] = df[['agent','company']].fillna(0.0)


## For the missing values in the country column, replace it with mode (value that appears most often)
df['country'].fillna(data.country.mode().to_string(), inplace=True)


## for missing children value, replace it with rounded mean value
df['children']=df['children'].fillna(0.0)

In [12]:
## convert datatype of these columns from float to integer
df[['children', 'company', 'agent']] = df[['children', 'company', 'agent']].astype('int64')

### 숙박 일수 1박 미만

In [13]:
# 총 숙박 일수가 1박 미만인 row 제거

# 총 숙박 일수 feature 생성
df['total_number_of_staying_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# 총 숙박 일수가 1박 이상인 항목만 선택
df = df[ df['total_number_of_staying_nights'] >= 1 ]
df = df.reset_index(drop=True)

In [14]:
# adr이 0 이하인 row 제거
df = df[ df['adr'] > 0 ].copy()

In [15]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,total_number_of_staying_nights
0,Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,1
1,Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,1
2,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2
3,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2
4,Resort Hotel,0,0,2015,July,27,1,0,2,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,0,0,0,Transient,107.0,0,0,Check-Out,2015-07-03,2


In [16]:
## Copy the dataframe
df_subset = df.copy()

In [17]:
## Remove reservation_status column
## because it tells us if booking was cancelled 
df_subset = df_subset.drop(['reservation_status'], axis=1)

## Make the new column which contain 1 if guest received the same room which was reserved otherwise 0
df_subset['Room'] = 0
df_subset.loc[ df_subset['reserved_room_type'] == df_subset['assigned_room_type'] , 'Room'] = 1


## Make the new column which contain 1 if the guest has cancelled more booking in the past
## than the number of booking he did not cancel, otherwise 0

df_subset['net_cancelled'] = 0
df_subset.loc[ df_subset['previous_cancellations'] > df_subset['previous_bookings_not_canceled'] , 'net_cancelled'] = 1

df_subset['reservation_status_date'] = pd.to_datetime(df_subset['reservation_status_date'])

df_subset['year'] = df_subset['reservation_status_date'].dt.year
df_subset['month'] = df_subset['reservation_status_date'].dt.month
df_subset['day'] = df_subset['reservation_status_date'].dt.day

df_subset = df_subset.drop(['booking_changes','assigned_room_type','reservation_status_date','distribution_channel'],axis=1)
# 'arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','arrival_date_month'

In [18]:
def transform(dataframe):
    
    
    ## Import LabelEncoder from sklearn
    from sklearn.preprocessing import LabelEncoder
    
    le = LabelEncoder()
    
    
    ## Select all categorcial features
    categorical_features = list(dataframe.columns[dataframe.dtypes == object])
    
    
    ## Apply Label Encoding on all categorical features
    dataframe[categorical_features]=dataframe[categorical_features].apply(lambda x: le.fit_transform(x))
    return dataframe

df_subset = transform(df_subset)

In [19]:
df_subset.var()

hotel                                 0.222716
is_canceled                           0.234348
lead_time                         11430.508914
arrival_date_year                     0.499289
arrival_date_month                   12.499659
arrival_date_week_number            184.281629
arrival_date_day_of_month            77.148962
stays_in_weekend_nights               0.989183
stays_in_week_nights                  3.572044
adults                                0.231569
children                              0.159791
babies                                0.009444
meal                                  1.138017
country                            2024.776043
market_segment                        1.520872
is_repeated_guest                     0.027086
previous_cancellations                0.720456
previous_bookings_not_canceled        2.091370
reserved_room_type                    2.864232
deposit_type                          0.113640
agent                             11500.539910
company      

In [20]:
df_subset['lead_time']=np.log(df_subset['lead_time']+1)
df_subset['agent'] = np.log(df_subset['agent'] + 1)
df_subset['company'] = np.log(df_subset['company'] + 1)
df_subset['adr'] = np.log(df_subset['adr'] + 1)
df_subset['country'] = np.log(df_subset['country'] + 1)
df_subset['days_in_waiting_list'] = np.log(df_subset['days_in_waiting_list'] + 1)
df_subset['month'] = np.log(df_subset['month'] + 1)
df_subset['day'] = np.log(df_subset['day'] + 1)
df_subset['arrival_date_week_number'] = np.log(df_subset['arrival_date_week_number'] + 1)
df_subset['arrival_date_day_of_month'] = np.log(df_subset['arrival_date_day_of_month'] + 1)
df_subset['arrival_date_month'] = np.log(df_subset['arrival_date_month'] + 1)

df_subset = df_subset.drop([])

In [21]:
df_subset.var()

hotel                             0.222716
is_canceled                       0.234348
lead_time                         2.520964
arrival_date_year                 0.499289
arrival_date_month                0.575158
arrival_date_week_number          0.439545
arrival_date_day_of_month         0.506808
stays_in_weekend_nights           0.989183
stays_in_week_nights              3.572044
adults                            0.231569
children                          0.159791
babies                            0.009444
meal                              1.138017
country                           0.465148
market_segment                    1.520872
is_repeated_guest                 0.027086
previous_cancellations            0.720456
previous_bookings_not_canceled    2.091370
reserved_room_type                2.864232
deposit_type                      0.113640
agent                             3.496853
company                           1.311311
days_in_waiting_list              0.507638
customer_ty

In [22]:
df_subset['adr'] = df_subset['adr'].fillna(value = df_subset['adr'].mean())

In [23]:
df_subset.shape

(117399, 33)

In [24]:
def data_split(df, label):
    
    from sklearn.model_selection import train_test_split

    X = df.drop(label, axis=1)
    Y = df[label]

    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.30)
    
    return x_train, x_test, y_train, y_test



x_train, x_test, y_train, y_test = data_split(df_subset, 'is_canceled')

from sklearn.impute import SimpleImputer

## default, imputing 'mean' value
imputer = SimpleImputer() 
x_train = imputer.fit_transform(x_train)
x_test = imputer.transform(x_test)

In [25]:
def train(x_train, y_train):
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    from sklearn.ensemble import GradientBoostingClassifier 
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from catboost import CatBoostClassifier

    #85퍼
    clf = DecisionTreeClassifier(random_state=0)
    clf.fit(x_train,y_train)


    #86퍼
    xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 500)
    xgb.fit(x_train, y_train)
    
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)

    #86퍼
    ada = AdaBoostClassifier(base_estimator = clf)
    ada.fit(x_train, y_train)

    #88 퍼
    rd_clf = RandomForestClassifier(n_estimators=300)
    rd_clf.fit(x_train, y_train)    

    #80퍼
    lr = LogisticRegression()
    lr.fit(x_train, y_train)

    cat = CatBoostClassifier(iterations=300)
    cat.fit(x_train, y_train)

    return clf, xgb,gb,ada,rd_clf,lr,cat


clf,xgb,gb,ada,rd_clf,lr,cat = train(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Learning rate set to 0.204182
0:	learn: 0.5470417	total: 76.2ms	remaining: 22.8s
1:	learn: 0.4756716	total: 118ms	remaining: 17.7s
2:	learn: 0.4305357	total: 145ms	remaining: 14.4s
3:	learn: 0.4044100	total: 172ms	remaining: 12.7s
4:	learn: 0.3818510	total: 203ms	remaining: 12s
5:	learn: 0.3529858	total: 229ms	remaining: 11.2s
6:	learn: 0.3391293	total: 258ms	remaining: 10.8s
7:	learn: 0.3133661	total: 287ms	remaining: 10.5s
8:	learn: 0.3048061	total: 312ms	remaining: 10.1s
9:	learn: 0.3006068	total: 339ms	remaining: 9.82s
10:	learn: 0.2825371	total: 367ms	remaining: 9.63s
11:	learn: 0.2771840	total: 398ms	remaining: 9.56s
12:	learn: 0.2662324	total: 430ms	remaining: 9.49s
13:	learn: 0.2564755	total: 459ms	remaining: 9.38s
14:	learn: 0.2383703	total: 485ms	remaining: 9.21s
15:	learn: 0.2266531	total: 513ms	remaining: 9.11s
16:	learn: 0.2048907	total: 539ms	remaining: 8.98s
17:	learn: 0.1959581	total: 564ms	remaining: 8.83s
18:	learn: 0.1859382	total: 593ms	remaining: 8.78s
19:	learn: 0

In [26]:
def Score(clf,x_train,y_train,x_test,y_test):
    train_score = clf.score(x_train,y_train)
    test_score = clf.score(x_test,y_test)

    print("========================================")
    print(f'Training Accuracy of our model is: {train_score}')
    print(f'Test Accuracy of our model is: {test_score}')
    print("========================================")
    
    
Score(clf,x_train,y_train,x_test,y_test)

Training Accuracy of our model is: 1.0
Test Accuracy of our model is: 0.9433560477001703


In [27]:
models = pd.DataFrame({
    'Model' : ['Decision Tree Classifier', 'Random Forest Classifier', 
               'XgBoost',  'Cat Boost', 'Gradient Boosting Classifier','Ada Boost Classifier', 'Linear regression'],

    'Score' : [clf.score(x_test,y_test),rd_clf.score(x_test,y_test), xgb.score(x_test,y_test), cat.score(x_test,y_test) , gb.score(x_test,y_test) , ada.score(x_test,y_test) , lr.score(x_test,y_test)]
})

models.sort_values(by = 'Score', ascending = False)

,Model,Score
3,Cat Boost,0.999006
2,XgBoost,0.998864
1,Random Forest Classifier,0.956502
0,Decision Tree Classifier,0.943356
5,Ada Boost Classifier,0.942362
4,Gradient Boosting Classifier,0.920585
6,Linear regression,0.786513
